In [98]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import operator
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
data = datasets.load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
Y = pd.DataFrame(data.target)

SS = StandardScaler()
X = pd.DataFrame(SS.fit_transform(X))

X.rename(columns={0: 'sepal length (cm)', 1: 'sepal width (cm)', 2: 'petal length (cm)', 3: 'petal width (cm)'}, inplace=True)
Y.rename(columns={0: 'target'}, inplace=True)

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

X_train.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

In [ ]:
knn_pipe = Pipeline([('knn', KNeighborsClassifier())])
params = [{'knn__n_neighbors': [3, 5, 7, 9],
         'knn__weights': ['uniform', 'distance'],
         'knn__leaf_size': [15, 20]}]

gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='accuracy',
                      cv=5)
gs_knn.fit(X_train, Y_train.values.ravel())
gs_knn.best_params_

{'knn__leaf_size': 15, 'knn__n_neighbors': 5, 'knn__weights': 'uniform'}

In [99]:
Y_pred = []
temp = []
negh = []

for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
      temp.append((np.linalg.norm(X_test.loc[i] - X_train.loc[j]), i, j))

  temp.sort(reverse=True,key=operator.itemgetter(0))

  for k in range(5):
    negh.append(Y_train.iloc[temp.pop()[2], 0])
  Y_pred.append(max(negh, key=negh.count))
  negh.clear()

cm = confusion_matrix(Y_test, Y_pred)
print(cm)
print(classification_report(Y_test, Y_pred, zero_division=1))

[[15  0  1]
 [ 0 17  1]
 [ 4  0  7]]
              precision    recall  f1-score   support

           0       0.79      0.94      0.86        16
           1       1.00      0.94      0.97        18
           2       0.78      0.64      0.70        11

    accuracy                           0.87        45
   macro avg       0.86      0.84      0.84        45
weighted avg       0.87      0.87      0.86        45

